# 钟摆的间谐运动

在这一节中，我们将重新创建一个涉及摆的有趣实验。如果我们创建了具有不同固有频率的一系列摆，并让其在相同的位置开始运动。我们将看到它们会在不同的频率产生振荡。但是，如果我们从系统中删除所有能量耗散，所有的摆最终将在其最初的位置“团聚”。

一般来说，这些“团聚”事件间的周期为是系统中所有摆周期的最小公倍数。我们可以通过选择摆的长度来实现一个特定的“团圆”周期。

## 钟摆模式

在本节中，我们可以像空间分布的传热的例子一样用组件数组来建立子系统模型。但在创建摆的数组之前，我们需要先有一个单摆的模型。
```Modelica
within ModelicaByExample.Subsystems.Pendula;
model Pendulum "A single individual pendulum"
  import Modelica.Mechanics.MultiBody.Parts;
  import Modelica.Mechanics.MultiBody.Joints;

  parameter Modelica.SIunits.Position x;
  parameter Modelica.SIunits.Mass m "Mass of mass point";
  parameter Modelica.SIunits.Angle phi "Initial angle";
  parameter Modelica.SIunits.Length L "String length";
  parameter Modelica.SIunits.Diameter d=0.01;

  Parts.Fixed ground(r={0,0,x}, animation=false)
    annotation (Placement(
        transformation(
        extent={{-10,-10},{10,10}},
        rotation=270, origin={0,60})));
  Parts.PointMass ball(m=m, sphereDiameter=5*d)
    annotation (Placement(transformation(extent={{-10,-90},{10,-70}})));
  Parts.BodyCylinder string(density=0, r={0,L,0}, diameter=d)
    annotation (Placement(transformation(
        extent={{-10,-10},{10,10}},
        rotation=90,
        origin={0,-30})));
  Joints.Revolute revolute(phi(fixed=true, start=phi),
    cylinderDiameter=d/2, animation=false)
    annotation (Placement(
        transformation(
        extent={{-10,-10},{10,10}},
        rotation=90,
        origin={0,20})));
equation
  connect(string.frame_a, ball.frame_a) annotation (Line(
      points={{0,-40},{0,-40},{0,-80}},
      color={95,95,95},
      thickness=0.5,
      smooth=Smooth.None));
  connect(revolute.frame_b, ground.frame_b) annotation (Line(
      points={{0,30},{0,40},{0,40},{0,50}},
      color={95,95,95},
      thickness=0.5,
      smooth=Smooth.None));
  connect(revolute.frame_a, string.frame_b) annotation (Line(
      points={{0,10},{0,10},{0,-20},{0,-20}},
      color={95,95,95},
      thickness=0.5,
      smooth=Smooth.None));
end Pendulum;
```

摆的组件显示如下：

<img src="./figures8/Pendulum.svg" width=800 align=center>


## 系统模型

现在，有了单独的摆模型，我们就可以建立钟摆组成的系统。如果我们想要一个有$n$个摆的系统，而系统的完整周期为$T$秒。而第$i$个摆的长度可以用如下方式计算：

$$l_i = g_n\frac{T}{2\pi(X+(n-i))}$$

其中$g_n$是地球的引力常数。$n$是钟摆的数量。$T$是该系统完整循环的周期。而$X$是最长的摆锤在$T$秒的振荡次数。

Modelica里我们可以建立上述系统，如下：
```Modelica
within ModelicaByExample.Subsystems.Pendula;
model System "A system of pendula"
  import Modelica.Constants.g_n;
  import Modelica.Constants.pi;

  parameter Integer n=15 "Number of pendula";
  parameter Modelica.SIunits.Position x[n] = linspace(0,(n-1)*0.05,n);
  parameter Modelica.SIunits.Time T = 54;
  parameter Modelica.SIunits.Time X = 30;
  parameter Modelica.SIunits.Length lengths[n] = { g_n*(T/(2*pi*(X+(n-i))))^2 for i in 1:n};
  parameter Modelica.SIunits.Angle phi0 = 0.5;

  Pendulum pendulum[n](x=x, each m=1, each phi=phi0, L=lengths)
    annotation (Placement(transformation(extent={{-10,-10},{10,10}})));
  inner Modelica.Mechanics.MultiBody.World world
    annotation (Placement(transformation(extent={{-80,-60},{-60,-40}})));
end System;
```

以下声明特别值得关注：
```Modelica
Pendulum pendulum[n](x=x, each m=1, each phi=phi0, L=lengths)
```
因为pendulum是有n个分量的数组，将有n个x、m、phi和L这些与摆相关的参数。举个例子，如果n=3，那么模型的x将有3个值：pendulum[1].x、pendulum[2].x和pendulum[3].x。在pendulum声明里，我们以不同方式对不同参数进行处理。对于m，我们使用each m=1为每个钟摆赋相同的值。然而，对于L（和x），我们提供数组以进行赋值。而lengths数组中的值使用前面介绍的钟摆长度公式来计算。我们将在本章后面更完整地讨论如何将修改应用到组件数组。

如果对上述系统进行仿真，我们可以得到各摆轨迹的解。所有摆在每过54秒后都会回到初始位置。

In [1]:
from IPython.display import Video
from IPython.display import display
vid = Video("./figures8/Experiment in Harmonic Motion.mp4")
display(vid)

## 结论

本节中，我们看到了如何才能使用、声明和修改组件数组。在本节的例子里，这允许我们指定系统内摆的数目并进行仿真。目的是观察这些摆在应用由前述方程式指定的不同长度时的特别行为。